# IE Function


In [ ]:
#| default_exp rgxlog.ie_function

In [ ]:
#| hide
from __future__ import annotations

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
from typing import Iterable, Callable, Union, Tuple, List, Sequence, Any
from spanner_workbench.rgxlog.primitive_types import DataTypes

In [ ]:
#| export
class IEFunction:
    """
    A class that contains all the functions that provide data
    needed for using a single information extraction function
    """

    def __init__(self,
            ie_function_def: Callable, # the user defined ie function implementation
            in_types: Sequence[DataTypes], # iterable of the input types to the function
            out_types: Union[List[DataTypes],Callable[[int], Sequence[DataTypes]]] # either a function (int->iterable) or an iterable
            ):
        self.ie_function_def = ie_function_def
        self.in_types = in_types
        self.out_types = out_types
    
    def ie_function(self, *args: Any) -> Iterable[Iterable[Union[str, int, Tuple[int, int]]]]:  # Tuple[int, int] represents a Span
        """
        The actual information extraction function that will be used
        the function must return a list of lists/tuples that represents the results, another option is to yield the
        tuples.

        currently the values inside the returned tuples can belong to three datatypes: string, integer and span
        string should be returned as a str instance
        an integer should be returned as an int instance
        a span could be returned either as a tuple of length 2, or as a datatypes.Span instance
        """
        output = self.ie_function_def(*args)
        return output

    def get_input_types(self) -> Sequence[DataTypes]:
        """
        @return: an iterable of the input types to the function.
        This function must be defined as it is used for type checking in semantic passes and execution.
        """
        return self.in_types

    def get_output_types(self, output_arity: int # expected output_arity
                        ) -> Sequence[DataTypes]: # given an expected output arity returns an iterable of the output types to the function
        """
        if the ie function cannot return an output of length output_arity, should return None.
        This function must be defined as it is used for type checking in semantic passes and execution.
        """

        if callable(self.out_types):
            return list(self.in_types) + list(self.out_types(output_arity - len(self.in_types)))

        # output is constant
        if not output_arity == len(list(self.in_types)) + len(list(self.out_types)):
            raise Exception("Output arity doesn't match the declared arity.")
        return list(self.in_types) + list(self.out_types)

    def get_meta_data(self) -> str:
        """
        @return: metadata about the ie function.
        """
        metadata = f"""Input types: {self.in_types}.\nOutput types: {self.out_types}"""
        return metadata

In [ ]:
show_doc(IEFunction.ie_function)

---

[source](https://github.com/loayshaqir1/spanner_workbench/blob/main/spanner_workbench/src/rgxlog_interpreter/src/rgxlog/engine/ie_function.py#L26){target="_blank" style="float:right; font-size:smaller"}

### IEFunction.ie_function

>      IEFunction.ie_function (*args:Any)

The actual information extraction function that will be used
the function must return a list of lists/tuples that represents the results, another option is to yield the
tuples.

currently the values inside the returned tuples can belong to three datatypes: string, integer and span
string should be returned as a str instance
an integer should be returned as an int instance
a span could be returned either as a tuple of length 2, or as a datatypes.Span instance

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| args | Any |  |
| **Returns** | **Iterable[Iterable[Union[str, int, Tuple[int, int]]]]** | **Tuple[int, int] represents a Span** |

In [ ]:
show_doc(IEFunction.get_input_types)

---

[source](https://github.com/loayshaqir1/spanner_workbench/blob/main/spanner_workbench/src/rgxlog_interpreter/src/rgxlog/engine/ie_function.py#L40){target="_blank" style="float:right; font-size:smaller"}

### IEFunction.get_input_types

>      IEFunction.get_input_types ()

@return: an iterable of the input types to the function.
This function must be defined as it is used for type checking in semantic passes and execution.

In [ ]:
show_doc(IEFunction.get_output_types)

---

[source](https://github.com/loayshaqir1/spanner_workbench/blob/main/spanner_workbench/src/rgxlog_interpreter/src/rgxlog/engine/ie_function.py#L47){target="_blank" style="float:right; font-size:smaller"}

### IEFunction.get_output_types

>      IEFunction.get_output_types (output_arity:int)

if the ie function cannot return an output of length output_arity, should return None.
This function must be defined as it is used for type checking in semantic passes and execution.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| output_arity | int | expected output_arity |
| **Returns** | **Sequence[DataTypes]** | **given an expected output arity returns an iterable of the output types to the function** |

In [ ]:
show_doc(IEFunction.get_meta_data)

---

[source](https://github.com/loayshaqir1/spanner_workbench/blob/main/spanner_workbench/src/rgxlog_interpreter/src/rgxlog/engine/ie_function.py#L62){target="_blank" style="float:right; font-size:smaller"}

### IEFunction.get_meta_data

>      IEFunction.get_meta_data ()

@return: metadata about the ie function.

::: {.callout-note collapse="true"}

##### Example

In [ ]:
def sample_ie_func1(x: int, y: str):
    yield (x, y, x + 1)

ie_func = IEFunction(sample_ie_func1, [DataTypes.integer, DataTypes.string], [DataTypes.integer, DataTypes.string, DataTypes.integer])
print(ie_func.get_meta_data())


Input types: [<DataTypes.integer: 2>, <DataTypes.string: 0>].
Output types: [<DataTypes.integer: 2>, <DataTypes.string: 0>, <DataTypes.integer: 2>]


In [ ]:
result = list(ie_func.ie_function(1, "example"))
print("result =",result)

result = [(1, 'example', 2)]


:::